### Data Super Heroes

1. Install Selenium and Scrapy

-   Selenium: to load the reviews.
-   Scrapy: to extract the relevant information.

In [ ]:
! pip install selenium
! pip install scrapy

2. Dowload Chrome Driver

3. Importing dependencies

In [2]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

4. Selenium test

In [19]:
driver = webdriver.Chrome()
url = 'https://www.imdb.com/title/tt15398776/reviews/?ref_=tt_ql_2'
time.sleep(1)
driver.get(url)
time.sleep(1)
print(driver.title)
time.sleep(1)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.PAGE_DOWN)
time.sleep(1)
body.send_keys(Keys.PAGE_DOWN)
time.sleep(1)
body.send_keys(Keys.PAGE_DOWN)

Oppenheimer (2023) - Oppenheimer (2023) - User Reviews - IMDb


5. Extract the review count

In [29]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace('.','').split(' ')[0]



3.183